activity limitation deficit for efi2 

In [ ]:

 CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_al` as (
 with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent,
  cast(numericvalue as bignumeric) as barthel_index
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from 
                     `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate,
    j.birth_date,
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for fracture snomed codes before care home entry date and after age of 55 years
  b as (
    select
    person_id,
    snomedcode,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID 
    from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist`
    where deficit = 'Activity limitation' and Otherinstructions IS NULL) 
      AND dateevent < first_episodestartdate),
#filter for barthel index below 19
  c as (
    select
    person_id,
    dateevent
    from a_master
    where snomedcode in(select SNOMEDCT_CONCEPTID from 
                        `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` 
                        where deficit = 'Activity limitation')
      AND dateevent < first_episodestartdate and barthel_index < 19),
#union distinct individuals identified from ctv3 and snomed ct codes together
  snomed_numeric as (
    select
    person_id,dateevent from b

    union distinct

    select person_id,dateevent
    from c)
#apply dummy variable
    select
    distinct person_id,
    case when dateevent IS NOT NULL then 'yes'
    end as activity_limitation
    from snomed_numeric)
  



